# Lecture 11 - Fast RL I

provided by [Stanford CS234](https://www.youtube.com/watch?v=FgzM3zpZ55o)

---

<div class="alert alert-block alert-info">
Table of Contents: <br>
    
<ul>
    <li>1. <a href="#1.-Introduction">Introduction</a>
    <li>2. <a href="#2.-Multi-Armed-Bandits">Multi-Armed Bandits</a>
        <ul>
            <li>2.1. <a href="#2.1.-Greedy-Algorithm">Greedy Algorithm</a></li>
            <li>2.2. <a href="#2.2.-Upper-Confidence-Bounds">Upper Confidence Bounds</a></li>
        </ul>
    </li>
    <li>3. <a href="#3.-Resource">Resource</a></li>
</ul>
</div>

# 1. Introduction

2 Categories:
* _Computationally efficiency_
    * takes a long time to compute something (an AV needs to calculate something fast as it is moving)
* _Sample efficiency_
    * sometimes experience/data is hard to gather
    
How do we evaluate our algorithm?
* how good is it?
* does it converge?
* how quickly does it converge?

We usually evaluate an algorithm based on its performance, but today we will evaluate it based on the amount of data it needs to make good decisions.

The next 3 lectures (including this one) will cover the following:
* __settings__: (bandits, MDPs, etc)
* __frameworks__: evaluation criteria for evaluating RL algorithms
* __approaches__: classes of algorithms for achieving particular evaluation criterias

Specifically, for today's lecture we will cover:
* setting: multi-armed bandits
* framework: regret
* approach: optimism under uncertainty
* framework: bayesian regret
* approach: probability matching/thompson sampling

# 2. Multi-Armed Bandits

* Multi-armed bandits is a tuple of $(\mathcal{A}, \mathcal{R})$
* $\mathcal{A}$ is a known set of $m$ actions (arms)
* $R^{a}(r) = \mathbb{P}[r~|~a]$ is an unknown probability distribution over rewards
* each step $t$, the agent selects an action $a_{t} \in \mathcal{A}$ (pulling an arm)
* environment produces a reward $r_{t} \sim \mathcal{R}^{a_{t}}$
* Goal: maximize cumulative reward $\sum_{\tau = 1}^{t}r_{\tau}$

$$
Q(a) = \mathbb{E}[r~|~a]\hspace{1em} (Eq.~1)\\
V^{*} = Q(a^{*}) = \underset{a \in \mathcal{A}}{max}Q(a) \hspace{1em} (Eq.~2)\\
l_{t} = \mathbb{E}[V^{*} - Q(a_{t})] \hspace{1em} (Eq.~3)\\
L_{t} = \mathbb{E}[\sum_{\tau = 0}^{t} V^{*} - Q(a_{\tau})] \hspace{1em} (Eq.~4)
$$

Eq. 1 is the expected reward (mean reward) given an action (we ignore states in the multi-armed bandit setting). Eq. 2: Take the action $a$ that yields the largest action-value. Eq. 3: opportunity loss (the difference between the optimal action-value at time step $t$ and the action you took). Eq. 4 is the total opportunity loss (the sum of regrets across all time steps for an episode).

$$
\Delta_{i} = V^{*} - Q(a_{i})\\
\begin{equation}
    \begin{split}
    L_{t} & = \mathbb{E}[\sum_{\tau = 1}^{t}V^{*} - Q(a_{\tau})]\\
    & = \sum_{a \in \mathcal{A}} \mathbb{E}[N_{t}(a)](v^{*} - Q(a))\\
    & = \sum_{a \in \mathcal{A}} \mathbb{E}[N_{t}(a)]\Delta_{a}\\
    \end{split}
\end{equation} \hspace{1em} (Eq.~5)
$$

$N_{t}(a)$ is the number of times action $a$ has been picked up to time step $t$.

By maximizing cumulative reward, we minimize total regret.

## 2.1. Greedy Algorithm

The simplest approach is the greedy algorithm.

$$
\hat{Q}_{t}(a) = \frac{1}{N_{t}(a)} \sum_{t = 1}^{T} r_{t} \mathcal{1}(a_{t} = a) \hspace{1em} (Eq.~6)\\
a^{*}_{t} = \underset{a \in \mathcal{A}}{argmax} \hat{Q}_{t}(a) \hspace{1em} (Eq.~7)\\
$$

A slightly more nuanced version of this is the $\epsilon$-greedy algorithm. It will select $a_{t} = \underset{a \in \mathcal{A}}{argmax} \hat{Q}_{t}(a)$ with probability $1 - \epsilon$ and a random action with probability $\epsilon$.

The problem with these is that they can get stuck in suboptimal actions forever.

## 2.2. Upper Confidence Bounds

Types of Regret bounds:
* __problem independent__: bound on regret is a function of $T$
* __problem dependent__: bound regret as a function of number of times we pull each arm and gap between reward and optimal action

From past work, we find that the lower bound is sublinear.

$$
\underset{t \rightarrow \infty}{lim} L_{t} \ge log(t) \sum_{a~|~\Delta_{a} > 0} \frac{\Delta_{a}}{D_{KL}(\mathcal{R}^{a}||\mathcal{R}^{a*})} \hspace{1em} (Eq.~8)\\
$$

In Upper Confidence Bounds (UCB),
* we estimate an upper confidence $U_{t}(a)$ for each action value such that $Q(a) \le U_{t}(a)$
* depends on number of times $N_{t}(a)$
* select action to maximize UCB: $a_{t} = \underset{a \in \mathcal{A}}{argmax}[U_{t}(a)]$

We leverage __Hoeffding's Inequality__:

Consider X to be an i.i.d. random variable in $[0, 1]$ from 1 to n. $\bar{X}_{n}$ to be the sample mean.
$$
\mathcal{P}[\mathcal{E}[X] > \bar{X}_{n} + u] \le exp(-2nu^{2}) \hspace{1em} (Eq.~9)\\
$$

We then derive the UCB equation for selecting an action at timestep $t$. 

$$
\begin{equation}
    \begin{split}
    U_{t}(a) = \hat{Q}(a) + \sqrt{\frac{2 log(t)}{N_{t}(a)}}\\
    a_{t} = \underset{a \in \mathcal{A}}{argmax}[U_{t}(a)]
    \end{split}
\end{equation} \hspace{1em} (Eq.~10)\\
$$

UCB achieves logarithmic asymptotic total regret:

$$
\underset{t \rightarrow \infty}{lim} L_{t} \le 8 log(t) \sum_{a~|~\Delta_{a} > 0} \Delta_{a} \hspace{1em} (Eq.~11)\\
$$

For an example of how UCB works refer to this: https://www.youtube.com/watch?v=FgmMK6RPU1c&t=507s&ab_channel=ritvikmath.

# 3. Resource

If you missed the link right below the title, I'm providing the resource here again along with the course website.

- [Stanford CS234](https://www.youtube.com/watch?v=FgzM3zpZ55o)
- [Course Website](http://web.stanford.edu/class/cs234/index.html)

This is a series of 15 lectures provided by Stanford.
